In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

In [2]:
traindir = '../input/train_files/'
testdir = '../input/example_test_files/'
sp_train_df = pd.read_csv(traindir+'stock_prices.csv')
sp_train_df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [6]:
stock = sp_train_df['SecuritiesCode'][:2]
stock

0    1301
1    1332
Name: SecuritiesCode, dtype: int64

In [9]:
train_sp_np = np.load(traindir + 'processed/sp_date.npy')
train_sp_np.shape

(2, 1202, 10)

In [30]:
dates = sp_train_df['Date'].unique()
inde = ['Datediff', 'Open', 'High', 'Low', 'Close', 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Target']
testdf = pd.DataFrame(train_sp_np[0], index=dates, columns=inde)
testdf

,Datediff,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
2017-01-04,0.0,2734.0,2755.0,2730.0,2742.0,31400.0,1.0,NaN,0.0,0.000730
2017-01-05,1.0,2743.0,2747.0,2735.0,2738.0,17900.0,1.0,NaN,0.0,0.002920
2017-01-06,2.0,2734.0,2744.0,2720.0,2740.0,19900.0,1.0,NaN,0.0,-0.001092
2017-01-10,6.0,2745.0,2754.0,2735.0,2748.0,24200.0,1.0,NaN,0.0,-0.005100
2017-01-11,7.0,2748.0,2752.0,2737.0,2745.0,9300.0,1.0,NaN,0.0,-0.003295
...,...,...,...,...,...,...,...,...,...,...
2021-11-29,1790.0,2975.0,2984.0,2951.0,2951.0,14300.0,1.0,NaN,0.0,0.003793
2021-11-30,1791.0,2953.0,2997.0,2900.0,2900.0,20500.0,1.0,NaN,0.0,0.007558
2021-12-01,1792.0,2909.0,2936.0,2909.0,2911.0,10600.0,1.0,NaN,0.0,0.016706
2021-12-02,1793.0,2949.0,2973.0,2933.0,2933.0,15200.0,1.0,NaN,0.0,-0.003689


In [31]:
ReferDataLen = 1000
data_dict = {}
for i in range(len(stock)):
    SecuritiesCode = stock[i]
    wholedata = train_sp_np[i]
    print(wholedata.shape)
    tempdict = {}
    tempdict['Referdata'] = wholedata[:ReferDataLen, :7]
    tempdict['Query_train'] = wholedata[ReferDataLen:, :7]
    tempdict['Target_train'] = wholedata[ReferDataLen:, 9]
    data_dict[SecuritiesCode] = tempdict
data_dict[1301]['Query_train'].shape
data_dict[1301]['Referdata'].shape

(1202, 10)
(1202, 10)


(1000, 7)